In [ ]:
!pip install pandas 
!pip install numpy 
!pip install spacy
!pip install -U scikit-learn
!pip install seaborn
!pip install transformers

In [18]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from utils.emfd import *
from utils.spacy_preprocessor import SpaCyPreProcessor
import pickle
from tqdm.notebook import tqdm
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [11]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
davidson = pd.read_csv("datasets/edited/mfrc_reduced.csv")

In [19]:
def get_scores(scores):
    score = dict()
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        score[l] = np.round(float(s), 4)

    return score


davidson["positive"] = 0
davidson["neutral"] = 0
davidson["negative"] = 0


for i in tqdm(davidson.index):
    text = davidson["text"][i]
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    dict_scores = get_scores(scores)
    davidson["positive"][i] = dict_scores["positive"]
    davidson["neutral"][i] = dict_scores["neutral"]
    davidson["negative"][i] = dict_scores["negative"]

davidson

  0%|          | 0/17886 [00:00<?, ?it/s]

/tmp/ipykernel_15789/4062835361.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  davidson["positive"][i] = dict_scores["positive"]
/tmp/ipykernel_15789/4062835361.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  davidson["neutral"][i] = dict_scores["neutral"]
/tmp/ipykernel_15789/4062835361.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  davidson["negative"][i] = dict_scores["negative"]


,text,subreddit,bucket,annotator,annotation,confidence,care.virtue,care.vice,fairness.virtue,fairness.vice,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_text,positive,neutral,negative
0,That particular part of the debate is especial...,europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,particular debate especially funny macron expl...,0.1067,0.3189,0.5743
1,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,r france pretty lively lingo usually delibera...,0.1703,0.5590,0.2707
2,TBH Marion Le Pen would be better. Closet fasc...,neoliberal,French politics,annotator03,Non-Moral,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,tbh marion le pen well closet fascist vs flamb...,0.0658,0.4460,0.4882
3,it really is a very unusual situation isn't it...,europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,unusual situation fillon affair influence vote...,0.0204,0.3472,0.6324
4,The Le Pen brand of conservatism and classical...,europe,French politics,annotator03,Authority,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,le pen brand conservatism classical right wing...,0.0455,0.7794,0.1751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,My job is actually ok and I work with loads of...,antiwork,US Politics,annotator00,Non-Moral,Not Confident,0,0,0,0,0,0,0,0,0,0,job actually ok work load nice people decent p...,0.0765,0.1741,0.7494
17882,Someone dying of a disease doesn't change that...,Conservative,US Politics,annotator00,Non-Moral,Not Confident,0,0,0,0,0,0,0,0,0,0,die disease change massively react lethal peop...,0.0100,0.1579,0.8321
17883,Some of these politicians and commentators tak...,politics,US Politics,annotator00,Equality,Not Confident,0,0,0,0,0,0,0,0,0,0,politician commentator take offense biden call...,0.0186,0.2494,0.7321
17884,"No, it's ""truth over facts! Wait where am I ag...",Conservative,US Politics,annotator00,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,truth fact wait biden,0.0470,0.4317,0.5214


In [ ]:
davidson.to_csv("datasets/edited/mfrc_sentiment_score.csv", index = False, header = True)